Este código consegue dar conta da maioria dos datasets testados.

Este experimento foi utilizado como uma forma de estudo, pois seu funcionamento depende da configuração dos arquivos de input selecionados - que neste caso foram gerados de um sistema específico.



In [8]:
import pandas as pd
import numpy as np
import time
import logging
from functools import wraps
import threading

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Implementação do decorador timeout usando threading
def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = [TimeoutError(f"A função {func.__name__} excedeu o limite de tempo de {seconds} segundos")]
            def worker():
                try:
                    result[0] = func(*args, **kwargs)
                except Exception as e:
                    result[0] = e
            thread = threading.Thread(target=worker)
            thread.start()
            thread.join(seconds)
            if thread.is_alive():
                thread.join()
                raise result[0]
            if isinstance(result[0], Exception):
                raise result[0]
            return result[0]
        return wrapper
    return decorator

# Funções auxiliares
def safe_to_datetime(value):
    try:
        return pd.to_datetime(value, errors='coerce')
    except:
        return pd.NaT

def safe_to_numeric(value):
    try:
        return pd.to_numeric(value, errors='coerce')
    except:
        return np.nan

def safe_mode(series):
    try:
        return series.mode().iloc[0]
    except:
        return None

@timeout(300)  # 5 minutos de timeout
def process_data():
    logging.info("Iniciando processamento dos dados...")
    
    # Carrega as duas planilhas do arquivo Excel
    arquivo = pd.read_excel('C\\Local Origem\\ARQUIVO.xlsx', sheet_name='ARQUIVO')
    razao = pd.read_excel('C\\Local Origem\\RAZAO.xlsx', sheet_name='RAZAO')
    
    logging.info("Planilhas carregadas com sucesso.")

    # Filtra os lançamentos onde 'VALORES FISCAIS ICMS' é igual a "1"
    arquivo_filtrado = arquivo[arquivo['VALORES FISCAIS ICMS'] == 1].copy()

    # Preenche os valores vazios com uma string para que a comparação funcione corretamente
    arquivo_filtrado.fillna(value='', inplace=True)
    razao.fillna(value='', inplace=True)

    # Converte as colunas de data para datetime, ignorando erros
    arquivo_filtrado['DT.EMIS.'] = pd.to_datetime(arquivo_filtrado['DT.EMIS.'], errors='coerce')
    razao['DATAEMISSAOSAIDA'] = razao['DATAEMISSAOSAIDA'].apply(safe_to_datetime)

    # Converte a coluna 'NOTA' para numérico, tratando erros
    razao['NOTA'] = razao['NOTA'].apply(safe_to_numeric)

    logging.info("Pré-processamento concluído.")

    # Converte as colunas 'VALOR ICMS.ITNF.IM' e 'TOMADA CRED. MONO' para numérico
    arquivo_filtrado['VALOR ICMS.ITNF.IM'] = pd.to_numeric(arquivo_filtrado['VALOR ICMS.ITNF.IM'], errors='coerce')
    arquivo_filtrado['TOMADA CRED. MONO'] = pd.to_numeric(arquivo_filtrado['TOMADA CRED. MONO'], errors='coerce')

    # Agrupa e soma os valores para a planilha ARQUIVO, incluindo a soma das duas colunas
    arquivo_agrupado = arquivo_filtrado.groupby('NUMERO NF').agg({
        'CFOP': 'first',
        'DESCRICAO UNIDADE ORIGEM': 'first',
        'DESCRICAO UNIDADE DESTINO': 'first',
        'DT.EMIS.': 'first',
        'VALOR ICMS.ITNF.IM': 'sum',
        'TOMADA CRED. MONO': 'sum',
        'TIPO NF': 'first'
    }).reset_index()
    
    # Calcula a soma das duas colunas
    arquivo_agrupado['ICMS Total Livro'] = arquivo_agrupado['VALOR ICMS.ITNF.IM'] + arquivo_agrupado['TOMADA CRED. MONO']
    
    arquivo_agrupado.columns = ['NUMERO NF', 'CFOP', 'DESCRICAO UNIDADE ORIGEM', 'DESCRICAO UNIDADE DESTINO', 'DT.EMIS.', 'VALOR ICMS.ITNF.IM', 'TOMADA CRED. MONO', 'TIPO NF', 'ICMS Total Livro']

    # Agrupa e soma os valores para a planilha RAZAO
    razao_agrupado = razao.groupby('NOTA', dropna=False).agg({
        'DATAEMISSAOSAIDA': safe_mode,
        'VL_MC': 'sum',
        'LANC_SET': 'first'
    }).reset_index()
    razao_agrupado.columns = ['NOTA', 'DATAEMISSAOSAIDA', 'Soma_VL_MC', 'LANC_SET']

    logging.info("Agrupamento concluído.")

    # Converte para string
    colunas_para_string = {
        'arquivo_agrupado': ['NUMERO NF', 'CFOP', 'DESCRICAO UNIDADE ORIGEM', 'DESCRICAO UNIDADE DESTINO', 'VALOR ICMS.ITNF.IM', 'TOMADA CRED. MONO', 'ICMS Total Livro', 'TIPO NF'],
        'razao_agrupado': ['NOTA', 'Soma_VL_MC', 'LANC_SET']
    }

    for df_name, colunas in colunas_para_string.items():
        df = locals()[df_name]
        for coluna in colunas:
            df[coluna] = df[coluna].astype(str)

    arquivo_agrupado['DT.EMIS.'] = arquivo_agrupado['DT.EMIS.'].dt.strftime('%d/%m/%y')
    razao_agrupado['DATAEMISSAOSAIDA'] = razao_agrupado['DATAEMISSAOSAIDA'].dt.strftime('%d/%m/%y')
    razao_agrupado['NOTA'] = razao_agrupado['NOTA'].str.replace('.0', '', regex=False)

    logging.info("Conversão de tipos concluída.")

    # Mescla as duas planilhas
    planilha_comparada = pd.merge(arquivo_agrupado, razao_agrupado, left_on='NUMERO NF', right_on='NOTA', how='outer', suffixes=('_arquivo', '_razao'))

    logging.info("Mesclagem das planilhas concluída.")

    # Verifica diferenças
    diferenca_icms = planilha_comparada['ICMS Total Livro'] != planilha_comparada['Soma_VL_MC']
    diferenca_nf = planilha_comparada['NUMERO NF'] != planilha_comparada['NOTA']

    # Preenche valores vazios
    planilha_comparada.fillna('-', inplace=True)

    # Condição especial
    condicao_especial = (planilha_comparada['ICMS Total Livro'] == '0.0') & planilha_comparada['Soma_VL_MC'].eq('') & planilha_comparada['NOTA'].eq('')

    # Prepara o DataFrame final
    resultados = planilha_comparada[['TIPO NF', 'NUMERO NF', 'CFOP', 'DESCRICAO UNIDADE ORIGEM', 'DESCRICAO UNIDADE DESTINO', 'DT.EMIS.', 'ICMS Total Livro', 'DATAEMISSAOSAIDA', 'NOTA', 'Soma_VL_MC', 'LANC_SET']].copy()
    resultados.columns = ['Tipo de Documento', 'Num Nota Livro', 'CFOP', 'Fornecedor', 'Destino', 'Data Livro', 'ICMS Total Livro', 'Data Razão', 'Num Nota Razão', 'ICMS Razão', 'Tipo de lçto Razão']
    resultados['Diferença no ICMS?'] = diferenca_icms
    resultados['Diferença número Nota?'] = diferenca_nf

    resultados.loc[condicao_especial, 'Diferença no ICMS?'] = False
    resultados.loc[condicao_especial, 'Diferença número Nota?'] = False

    logging.info("Processamento de dados concluído.")
    
    return resultados

if __name__ == "__main__":
    try:
        resultados = process_data()
        
        # Especifique o caminho do arquivo Excel de destino
        caminho_destino = 'C:\\Local Destino\\arquivo_resultado.xlsx'

        # Exporte o DataFrame para um arquivo Excel
        resultados.to_excel(caminho_destino, index=False)

        logging.info(f"Arquivo Excel gerado com sucesso em: {caminho_destino}")
    except TimeoutError as te:
        logging.error(f"Timeout: {str(te)}")
    except Exception as e:
        logging.error(f"Erro durante o processamento: {str(e)}")

2024-10-11 15:30:50,251 - INFO - Iniciando processamento dos dados...
2024-10-11 15:31:14,143 - INFO - Planilhas carregadas com sucesso.
2024-10-11 15:31:14,207 - INFO - Pré-processamento concluído.
2024-10-11 15:31:14,510 - INFO - Agrupamento concluído.
2024-10-11 15:31:14,556 - INFO - Conversão de tipos concluída.
2024-10-11 15:31:14,567 - INFO - Mesclagem das planilhas concluída.
2024-10-11 15:31:14,578 - INFO - Processamento de dados concluído.
2024-10-11 15:31:15,442 - INFO - Arquivo Excel gerado com sucesso em: G:\Fiscal\Conciliação\arquivo_resultado.xlsx
